In [3]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import glob
import shutil
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.track as track
import thor.option as option
import thor.tag as tag
import thor.visualize as visualize

notebook_name = "gridrad_demo.ipynb"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
# Parent directory for saving outputs
base_local = Path.home() / "THOR_output"
start = "2010-01-20T18:00:00"
end = "2010-01-20T19:00:00"

# Create the data_options dictionary
converted_options = {"save": True, "load": False, "parent_converted": None}
gridrad_options = data.gridrad.gridrad_data_options(
    start=start, end=end, converted_options=converted_options
)
era5_pl_options = data.era5.data_options(start=start, end=end)
era5_sl_options = data.era5.data_options(
    start=start, end=end, data_format="single-levels"
)
data_options = option.consolidate_options(
    [gridrad_options, era5_pl_options, era5_sl_options]
)

dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, filename="gridrad_era5")

# Create the grid_options dictionary using the first file in the cpol dataset
grid_options = grid.create_options(
    name="geographic", regrid=False, altitude_spacing=None, geographic_spacing=None
)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, filename="gridrad_geographic")

# Create the tag_options dictionary
era5_pl_tag_options = data.era5.tag_options()
era5_sl_tag_options = data.era5.tag_options(dataset="era5_sl")
tag_options = option.consolidate_options([era5_pl_tag_options, era5_sl_tag_options])
tag.save_tag_options(tag_options, filename="era5")

# Create the track_options dictionary
track_options = option.mcs(
    dataset="gridrad",
    tags=["era5_pl", "era5_sl"],
    global_flow_margin=70,
    unique_global_flow=False,
)
# Disable tracking/matching for cell and anvil objects
track_options[0]["cell"]["tracking"] = {"method": None}
track_options[0]["anvil"]["tracking"] = {"method": None}
option.check_options(track_options)
option.save_track_options(track_options, filename="gridrad_mcs")

# Create the display_options dictionary
cell_vis_options = visualize.option.runtime_options(
    "cell", save=True, style="presentation", figure_types=["mask"]
)
anvil_vis_options = visualize.option.runtime_options(
    "anvil", save=True, style="presentation", figure_types=["mask"]
)
mcs_vis_options = visualize.option.runtime_options(
    "mcs", save=True, style="presentation", figure_types=["mask", "match"]
)
visualize_options = {
    "cell": cell_vis_options,
    "anvil": anvil_vis_options,
    "mcs": mcs_vis_options,
}
# visualize_options = None
visualize.option.save_display_options(visualize_options, filename="runtime_mcs")

2024-08-27 14:54:25,447 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/data_options/gridrad_era5.yaml
2024-08-27 14:54:25,497 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/grid_options/gridrad_geographic.yaml
2024-08-27 14:54:25,506 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/tag_options/era5.yaml
2024-08-27 14:54:25,513 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/track_options/gridrad_mcs.yaml
2024-08-27 14:54:25,544 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/visualize_options/runtime_mcs.yaml


In [66]:
import cProfile
import pstats

In [67]:
profiler = cProfile.Profile()
profiler.enable()

In [68]:
output_directory = base_local / "runs/gridrad_demo"
if output_directory.exists():
    shutil.rmtree(output_directory)
times = data.utils.generate_times(data_options["gridrad"])
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    tag_options,
    visualize_options,
    output_directory=base_local / "runs/gridrad_demo",
)


2024-08-27 14:54:27,299 - thor.track - DEBUG - Beginning simultaneous tracking.
2024-08-27 14:54:27,364 - thor.track - INFO - Processing 2010-01-20T18:00:00.
2024-08-27 14:54:27,366 - thor.data.gridrad - DEBUG - Updating gridrad dataset for 2010-01-20T18:00:00.
2024-08-27 14:54:27,367 - thor.data.gridrad - DEBUG - Converting gridrad data from nexrad_3d_v4_2_20100120T180000Z.nc
2024-08-27 14:54:27,466 - thor.data.gridrad - DEBUG - Filtering GridRad data
2024-08-27 14:54:28,126 - thor.data.gridrad - DEBUG - Removing clutter from the GridRad data
2024-08-27 14:54:28,185 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-08-27 14:54:28,436 - thor.data.gridrad - DEBUG - Removing low level clutter from the GridRad data
2024-08-27 14:54:28,678 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-08-27 14:54:28,914 - thor.data.gridrad - DEBUG - Restructuring GridRad dataset.
2024-08-27 14:54:29,303 - thor.track - DEBUG - Processing hierarchy level 

In [69]:
profiler.disable()
stats = pstats.Stats(profiler).sort_stats('cumtime')
stats.print_stats(50)

         2240378 function calls (2183716 primitive calls) in 21.104 seconds

   Ordered by: cumulative time
   List reduced from 2724 to 50 due to restriction <50>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        8    0.000    0.000   21.095    2.637 /home/ewan/miniconda3/envs/THOR/lib/python3.10/asyncio/base_events.py:1832(_run_once)
       16    0.000    0.000   20.736    1.296 /home/ewan/miniconda3/envs/THOR/lib/python3.10/asyncio/events.py:78(_run)
       16    0.000    0.000   20.736    1.296 {method 'run' of '_contextvars.Context' objects}
        4    0.000    0.000   20.735    5.184 /home/ewan/miniconda3/envs/THOR/lib/python3.10/site-packages/ipykernel/kernelbase.py:522(process_one)
        2    0.000    0.000   20.735   10.367 /home/ewan/miniconda3/envs/THOR/lib/python3.10/site-packages/ipykernel/kernelbase.py:536(dispatch_queue)
        2    0.000    0.000   20.735   10.367 /home/ewan/miniconda3/envs/THOR/lib/python3.10/site-packages/ipykernel/